In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import ADASYN


In [5]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import ADASYN

# -----------------------------
# 1️⃣ Load and clean dataset
# -----------------------------
dataset = pd.read_csv(r'D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv', index_col=0)
cleaned_dataset = dataset.dropna(subset=['clean_comment', 'category'])

X_raw = cleaned_dataset['clean_comment']
y_raw = cleaned_dataset['category'].map({-1: 2, 0: 0, 1: 1})  # remap labels for multi-class

# -----------------------------
# 2️⃣ Train-test split
# -----------------------------
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# -----------------------------
# 3️⃣ TF-IDF vectorization
# -----------------------------
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
X_train_tfidf = tfidf.fit_transform(X_train_raw)
X_test_tfidf  = tfidf.transform(X_test_raw)

# -----------------------------
# 4️⃣ Apply ADASYN on training set
# -----------------------------
adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_tfidf, y_train)

# -----------------------------
# 5️⃣ Base learners
# -----------------------------
lightgbm_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric='multi_logloss',
    reg_alpha=0.1,
    reg_lambda=0.1,
    learning_rate=0.08081298097796712,
    n_estimators=367,
    max_depth=20,
    random_state=42
)

xgb_model = XGBClassifier(
    objective='multi:softprob',
    num_class=3,
    eval_metric='mlogloss',
    learning_rate=0.09087294404864565,
    max_depth=10,
    n_estimators=264,
    reg_alpha=0.1,
    reg_lambda=0.1,
    use_label_encoder=False,
    random_state=42
)

# -----------------------------
# 6️⃣ Meta-learner: Logistic Regression
# -----------------------------
meta_learner = LogisticRegression(
    C=0.5345867446632488,
    penalty='l1',
    solver='liblinear',  # required for l1
    multi_class='ovr',
    max_iter=1000
)

# -----------------------------
# 7️⃣ Stacking classifier
# -----------------------------
stacking_model = StackingClassifier(
    estimators=[
        ('lightgbm', lightgbm_model),
        ('xgboost', xgb_model)
    ],
    final_estimator=meta_learner,
    cv=5,
    stack_method='predict_proba',  # use probabilities for meta-learner
    n_jobs=1  # sequential to avoid XGBoost weight errors
)

# -----------------------------
# 8️⃣ Train the stacking model
# -----------------------------
stacking_model.fit(X_train_resampled, y_train_resampled)

# -----------------------------
# 9️⃣ Predict and evaluate
# -----------------------------
y_pred = stacking_model.predict(X_test_tfidf)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))


c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.069514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100189
[LightGBM] [Info] Number of data points in the train set: 37479, number of used features: 975
[LightGBM] [Info] Start training from score -1.091037
[LightGBM] [Info] Start training from score -1.088815
[LightGBM] [Info] Start training from score -1.116216
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057284 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 84449
[LightGBM] [Info] Number of data points in the train set: 29983, number of used features: 975
[LightGBM] [Info] Start training from score -1.091070
[LightGBM] [Info] Start training from score -1.088788
[LightGBM] [Info] Start training from score -1.116209
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.054635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83523
[LightGBM] [Info] Number of data points in the train set: 29983, number of used features: 973
[LightGBM] [Info] Start training from score -1.090971
[LightGBM] [Info] Start training from score -1.088888
[LightGBM] [Info] Start training from score -1.116209
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.054394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83507
[LightGBM] [Info] Number of data points in the train set: 29984, number of used features: 970
[LightGBM] [Info] Start training from score -1.091004
[LightGBM] [Info] Start training from score -1.088822
[LightGBM] [Info] Start training from score -1.116243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\Users\apoor\anaconda3\Lib\site-packages\xgboost\sklearn.py:1395: User

Classification Report:

              precision    recall  f1-score   support

           0       0.76      0.94      0.84      2529
           1       0.87      0.76      0.81      3154
           2       0.70      0.62      0.66      1650

    accuracy                           0.79      7333
   macro avg       0.78      0.77      0.77      7333
weighted avg       0.79      0.79      0.79      7333

